# Accelerate Inference of Sparse Transformer Models with OpenVINO™ and 4th Gen Intel&reg; Xeon&reg; Scalable Processors
This tutorial demonstrates how to improve performance of sparse Transformer models with [OpenVINO](https://docs.openvino.ai/) on 4th Gen Intel® Xeon® Scalable processors. 

The tutorial downloads a BERT-base model which has been quantized, sparsified, and tuned for SST2 datasets using [Optimum-Intel](https://github.com/huggingface/optimum-intel). It demonstrates the inference performance advantage on 4th Gen Intel&reg; Xeon&reg; Scalable Processors by running it with [Sparse Weight Decompression](https://docs.openvino.ai/latest/openvino_docs_OV_UG_supported_plugins_CPU.html#sparse-weights-decompression), a runtime option that siezes model sparsity for efficiency. The notebook consists of the following steps:

- Install prerequisites
- Download and quantize sparse BERT model from a public source using the OpenVINO integration with Hugging Face Optimum.
- Compare sparse 8-bit vs. dense 8-bit inference performance.


## Prerequisites

In [1]:
!pip install optimum[openvino] openvino-dev datasets

## Imports

In [2]:
import shutil
from pathlib import Path

from optimum.intel.openvino import OVModelForSequenceClassification
from transformers import AutoTokenizer, pipeline
from huggingface_hub import hf_hub_download

/root/vchua/miniconda3/envs/ov-nb/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Download a quantized sparse the model using Hugging Face Optimum API

The first step is to download a quantized sparse transformers which has been translated to OpenVINO IR. Then, it will be put through a classification, for simple validation of a working downloaded model. 

In [3]:
# The following model has been quantized, sparsified using Optimum-Intel 1.7 which is enabled by OpenVINO and NNCF
# for reproducibility, refer https://huggingface.co/OpenVINO/bert-base-uncased-sst2-int8-unstructured80
model_id = "OpenVINO/bert-base-uncased-sst2-int8-unstructured80"

# The following two steps will set up the model and download them to HF Cache folder
ov_model = OVModelForSequenceClassification.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Let's take the model for a spin!
sentiment_classifier = pipeline("text-classification", model=ov_model, tokenizer=tokenizer)

text = "He's a dreadful magician."
outputs = sentiment_classifier(text)

print(outputs)

[{'label': 'negative', 'score': 0.9981877207756042}]


For benchmarking, we will use OpenVINO's benchmark application and let's put the IRs into a single folder.

In [4]:
# create a folder
quantized_sparse_dir = Path("bert_80pc_sparse_quantized_ir")
quantized_sparse_dir.mkdir(parents=True, exist_ok=True)

# following return path to specified filename in cache folder and download if not found
ov_ir_xml_path = hf_hub_download(repo_id=model_id, filename="openvino_model.xml")
ov_ir_bin_path = hf_hub_download(repo_id=model_id, filename="openvino_model.bin")

# copy IRs to our folder
shutil.copy(ov_ir_xml_path, quantized_sparse_dir)
shutil.copy(ov_ir_bin_path, quantized_sparse_dir)                                

'bert_80pc_sparse_quantized_ir/openvino_model.bin'

## Benchmark quantized dense inference performance
Benchmark dense inference performance using parallel execution on four CPU cores to simulate a small instance in the cloud infrastructure. Sequense length is set to 16 which is common for multiple use cases, e.g. conversational AI.

In [5]:
# Dump benchmarking config for dense inference
with open(f"{quantized_sparse_dir}/perf_config.json", "w") as outfile:
    outfile.write(
        """
        {
            "CPU": {"NUM_STREAMS": 4, "INFERENCE_NUM_THREADS": 4}
        }
        """
    )

In [6]:
!(benchmark_app \
  -m bert_80pc_sparse_quantized_ir/openvino_model.xml \
  -shape "input_ids[1,16],attention_mask[1,16],token_type_ids[1,16]" \
  -load_config bert_80pc_sparse_quantized_ir/perf_config.json)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[ WARNING ] Performance hint was not explicitly specified in command line. Device(CPU) performance hint will be set to THROUGHPUT.
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 130.04 ms
[ INFO ] Original mo

## Benchmark quantized sparse inference performance

To enable sparse weight decompression feature, users can add it to runtime config like below. `CPU_SPARSE_WEIGHTS_DECOMPRESSION_RATE` honors value between 0.5 and 1.0, it is a layer-level sparsity threshold for which a layer will be enabled.

In [7]:
# Dump benchmarking config for dense inference
# "CPU_SPARSE_WEIGHTS_DECOMPRESSION_RATE" controls minimum sparsity rate for weights to consider 
# for sparse optimization at the runtime.
with open(f"{quantized_sparse_dir}/perf_config_sparse.json", "w") as outfile:
    outfile.write(
        """
        {
            "CPU": {"NUM_STREAMS": 4, "INFERENCE_NUM_THREADS": 4, "CPU_SPARSE_WEIGHTS_DECOMPRESSION_RATE": 0.75}
        }
        """
    )

In [8]:
!(benchmark_app \
  -m bert_80pc_sparse_quantized_ir/openvino_model.xml \
  -shape "input_ids[1,16],attention_mask[1,16],token_type_ids[1,16]" \
  -load_config bert_80pc_sparse_quantized_ir/perf_config_sparse.json)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[ WARNING ] Performance hint was not explicitly specified in command line. Device(CPU) performance hint will be set to THROUGHPUT.
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 142.44 ms
[ INFO ] Original mo

## When this might be helpful

This feauture can improve inference performance for models with sparse weights in the scenarios when the model is deployed to handle multiple requests in parallel asyncronously. It is especially helpful in the case of small sequence length, e.g. 32 and lower.

For more details about asynchronous inference with OpenVINO please refer to the following documentation:
- [Deployment Optimization Guide](https://docs.openvino.ai/latest/openvino_docs_deployment_optimization_guide_common.html#doxid-openvino-docs-deployment-optimization-guide-common-1async-api)
- [Inference Request API](https://docs.openvino.ai/latest/openvino_docs_OV_UG_Infer_request.html#doxid-openvino-docs-o-v-u-g-infer-request-1in-out-tensors)